## Загрузим нужные библиотеки

In [1]:
!pip install catboost
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import uniform
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf 
import numpy as np
import pickle
from sklearn.naive_bayes import MultinomialNB
import catboost as cb
%matplotlib inline

2022-11-11 10:23:42.391016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 10:23:42.530830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-11 10:23:42.530858: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-11 10:23:42.567042: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 10:23:43.181456: W tensorflow/stream_executor/platform/de

In [2]:
df = pd.read_csv("train_frame.csv",low_memory=False)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label,is_wrong,total_mean
0,6818780,26,17235,18,1,1,0.0,2,0,0.0,...,58950.0,779126.0,8290896.0,0,0,0,0,0,0,0.093974
1,9907176,20,23,4,1,1,0.0,1,0,0.0,...,83318932.0,132175590.0,136819803.0,0,0,0,0,0,0,0.966056
2,3304275,15,16331,19,1,1,0.0,2,0,0.0,...,3233068.0,6479360.0,52708071.0,0,1,0,0,0,1,0.122929
3,9020937,16,6325,19,1,1,0.0,2,0,0.0,...,653280.0,2714208.0,19562334.0,0,0,0,0,0,0,0.138747
4,3082311,17,17315,18,1,1,0.0,2,0,1.0,...,27911.0,344830.0,4719186.0,0,0,0,0,0,0,0.073070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,19,23,4,1,1,0.0,1,0,0.0,...,116432632.0,180702765.0,188407812.0,0,0,1,0,0,1,0.959104
5999996,2234489,19,4697,19,1,1,0.0,2,0,0.0,...,144063.0,1911433.0,15582018.0,0,0,0,0,0,0,0.122669
5999997,4304572,24,18,19,3,1,0.0,1,0,2.0,...,10648.0,60624000.0,75592387.0,0,0,0,0,0,0,0.801986
5999998,6550634,15,6,19,3,1,0.0,1,0,0.0,...,4972424.0,20063762.0,39988530.0,0,1,0,0,0,1,0.501738


In [3]:
df['label'].value_counts()

0    5829020
1     170980
Name: label, dtype: int64

Обьединим список не нужных строк с списком строк типа object

In [4]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [5]:
col_obj

['label', 'id']

In [6]:

X = df.drop(col_obj, axis = 1)
y = df["label"]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)

In [8]:
sm = SMOTE(random_state=42)

X, y = sm.fit_resample(X, y)

In [9]:
X_train.shape

(5100000, 29)

## Обучение модели

scaler = RobustScaler()
#scaler.fit(X)
# logistic = OneVsRestClassifier(RandomForestClassifier(max_depth=22,n_jobs=-1))
#logistic=GaussianNB()
#pipe = Pipeline([('normalizer', scaler), ('classifier', logistic)])
#pipe.fit(X_train, y_train)

clf1=HistGradientBoostingClassifier(learning_rate=0.1, max_depth= 8, min_samples_leaf=100)
clf2=RandomForestClassifier(max_depth=15,n_jobs=9)
eclf1 = VotingClassifier([('gb', clf2),('rf', clf2)], voting='soft',weights=[1,1.5])
pipe = Pipeline([('normalizer', scaler), ('classifier', eclf1)])
pipe.fit(X_train, y_train)

pool_train = cb.Pool(X, y)
params = {                  # 'task_type':'GPU',
                            'iterations':1200,
                            'depth':8,
                            'random_state':2,
                            'learning_rate':0.01,
                            'eval_metric':'Recall',
                            'loss_function':'MultiCrossEntropy'}
clf_with_aux = cb.CatBoostClassifier(**params)
clf_with_aux.fit(pool_train)


predcat = clf_with_aux.predict_proba(X_test)
voting = pipe.predict_proba(X_test)


y_pred = ((voting + predcat) / 2).argmax(axis=1)


print(classification_report(y_test, y_pred))



parametrs = {
              'max_depth': range (1,35),
            }
grid = GridSearchCV(RandomForestClassifier(), parametrs, scoring='roc_auc', cv=2,n_jobs=9) 
grid.fit(X_test, y_test) 
grid.best_params_



pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipe, file)

with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=2,   # use any cross validation technique 
                 verbose=1, 
                 scoring='roc_auc') 
gs_NB.fit(X_train, y_train)

gs_NB.best_params_


gb_grid_params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [4, 6, 8],
              'min_samples_leaf': [20, 50,100,150],
              #'max_features': [1.0, 0.3, 0.1] 
              }
print(gb_grid_params)

gb_gs = HistGradientBoostingClassifier()

clf = GridSearchCV(gb_gs,
                               gb_grid_params,
                               cv=2,
                               scoring='roc_auc',
                               verbose = 3, 
                               n_jobs=2);
clf.fit(X_train, y_train);
clf.best_params_

# Нейронка

from keras import models
from keras import layers
import tensorflow as tf 
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1000,activation='sigmoid',input_shape=(29,)))
    model.add(layers.Dense(2,activation='sigmoid'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',# Вы также можете указать параметры для оптимизатора через optimizer = optimizer.RMSprop (lr = 0.001)
                  loss='binary_crossentropy', # Эквивалент потерь = loss.binary_crossentropy
                  metrics=[tf.keras.metrics.AUC()]) # Эквивалентно метрике = [metircs.binary_accuracy]
    return model
model = build_model()


history = model.fit(X, y,
                    epochs=2, # Итерировать 20 раз по полному набору данных
                    batch_size=512,
                    class_weight={1:1,0:0.9},
                    validation_data=(X_test, y_test))

predictions = model.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

In [21]:
from keras_tuner import RandomSearch
from keras_tuner import Objective
from keras import models
from keras import layers
import tensorflow as tf 
def build_model(hyperparams):
    model = models.Sequential()
    model.add(layers.Input(shape=(X.shape[1],)))
    model.add(layers.Dense(units=hyperparams.Int("units_l1", 16, 50, step=16),
                           use_bias=hyperparams.Boolean("bias_l1"),
                           activation=hyperparams.Choice("act_l1", ["relu", "tanh",'sigmoid'])
                          ))
    model.add(layers.Dense(units=hyperparams.Int("units_l2", 16, 50, step=16),
                           use_bias=hyperparams.Boolean("bias_l2"),
                           activation=hyperparams.Choice("act_l2", ["relu", "tanh",'sigmoid'])
                          ))
    model.add(layers.Dense(1))

    optim=hyperparams.Choice("optimizer",["sgd","rmsprop","adam"])
    model.compile(optim, loss="binary_crossentropy", metrics=["mean_squared_error",tf.keras.metrics.AUC()])

    return model
tuner1 =  RandomSearch(hypermodel=build_model,
                      objective="binary_crossentropy",
                      #objective=Objective(name="val_mean_squared_error",direction="min"),
                      max_trials=100,
                      #seed=123,
                      project_name="Regression",
                      overwrite=True
                    )

tuner1.search(X_train, y_train, batch_size=512, epochs=3, validation_data=(X_test, y_test))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
48                |?                 |units_l1
True              |?                 |bias_l1
relu              |?                 |act_l1
48                |?                 |units_l2
False             |?                 |bias_l2
tanh              |?                 |act_l2
adam              |?                 |optimizer

Epoch 1/3
9961/9961 [==============================] - 27s 3ms/step - loss: 0.4512 - mean_squared_error: 1.8911 - auc: 0.4994 - val_loss: 0.4396 - val_mean_squared_error: 2.1409 - val_auc: 0.5000
Epoch 2/3
9961/9961 [==============================] - 26s 3ms/step - loss: 0.4396 - mean_squared_error: 2.3565 - auc: 0.5000 - val_loss: 0.4396 - val_mean_squared_error: 2.3751 - val_auc: 0.5000
Epoch 3/3
9961/9961 [==============================] - 27s 3ms/step - loss: 0.4396 - mean_squared_error: 2.3749 - auc: 0.5000 - val_loss: 0.4396 - val_mean_squared_error: 2.3751 - val_auc: 0.5000


KeyError: 'binary_crossentropy'

In [11]:
best_params = tuner1.get_best_hyperparameters()

best_params[0].values

{'units_l1': 48,
 'bias_l1': True,
 'act_l1': 'sigmoid',
 'units_l2': 32,
 'bias_l2': False,
 'act_l2': 'sigmoid',
 'optimizer': 'adam'}

In [12]:
best_model = tuner1.get_best_models()[0]

best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                1440      
                                                                 
 dense_1 (Dense)             (None, 32)                1536      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In [20]:
predictions = best_model.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

28125/28125 [==============================] - 22s 766us/step


/home/gennalll/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gennalll/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gennalll/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


нейронка               precision    recall  f1-score   support

           0       0.97      1.00      0.99    874353
           1       0.00      0.00      0.00     25647

    accuracy                           0.97    900000
   macro avg       0.49      0.50      0.49    900000
weighted avg       0.94      0.97      0.96    900000



# Нейронка 2

In [13]:
!pip install scikeras[tensorflow]
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=29, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
	return model

estimator = KerasClassifier(model=create_baseline, epochs=1, batch_size=512, verbose=0)
kfold = StratifiedKFold(n_splits=41, shuffle=True, random_state=42)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

model2=create_baseline()
model2.fit(X, y)
model2.predict(X_test)


predictions = model2.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

# Предсказание 

In [14]:
pred_data=pd.read_csv('test_dataset_test.csv',low_memory=False)


In [15]:
df=pred_data

In [16]:
df["priority"] = pd.Categorical(df["priority"])
df["priority"].astype('category').cat.codes
df["priority"] = df["priority"].cat.codes
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes
df["oper_type + oper_attr"] = pd.Categorical(df["oper_type + oper_attr"])
df["oper_type + oper_attr"].astype('category').cat.codes
df["oper_type + oper_attr"] = df["oper_type + oper_attr"].cat.codes
df["index_oper"] = pd.Categorical(df["index_oper"])
df["index_oper"].astype('category').cat.codes
df["index_oper"] = df["index_oper"].cat.codes
df["type"] = pd.Categorical(df["type"])
df["type"].astype('category').cat.codes
df["type"] = df["type"].cat.codes
df["is_privatecategory"] = pd.Categorical(df["is_privatecategory"])
df["is_privatecategory"].astype('category').cat.codes
df["is_privatecategory"] = df["is_privatecategory"].cat.codes
df["name_mfi"] = pd.Categorical(df["name_mfi"])
df["name_mfi"].astype('category').cat.codes
df["name_mfi"] = df["name_mfi"].cat.codes
df["weight"] = round(df["weight"]/1000)
df['is_wrong']=df['is_wrong_sndr_name']+df['is_wrong_rcpn_name']+df['is_wrong_phone_number']+df['is_wrong_address']
df['total_mean']=df['total_qty_over_index_and_type']/df['total_qty_over_index']

In [17]:
df = df.fillna(0)


In [18]:
df=df.drop(['id'], axis = 1)

pred_data['label']=pipe.predict(df)

pred_data[['id','label']].to_csv('bestmodel.csv', index=False)

# Нейро 

In [19]:
predictions = model.predict(df)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
pred_data['label']=pred
pred_data[['id','label']].to_csv('bestmodelnero.csv', index=False)

NameError: name 'model' is not defined